## Textos de videos

In [11]:
# Instalaciones necesarias
!apt-get update
!pip install selenium webdriver-manager flask youtube-transcript-api
!pip install langdetect

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,901 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,920 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restri

In [16]:
# Ruta origen
url_videos = "https://boardgamegeek.com/boardgame/199561/sagrada/videos/all?pageid=1&sort=hot"

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import csv
import os
import re

# Iniciar Selenium
def start_driver(url, delay=5):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    service = Service('/usr/bin/chromedriver')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    print(f"Esperando {delay} segundos para que cargue la página...")
    time.sleep(delay)
    return driver

# Extraer vídeos
def get_youtube_video_ids_from_bgg(driver, url_base):
    driver.get(url_base)
    time.sleep(5)
    print("Cargando enlaces internos de videos...")

    # Buscar todos los enlaces a páginas de videos
    links = driver.find_elements(By.CSS_SELECTOR, 'a[href^="/video/"]')
    video_page_urls = list(set([link.get_attribute("href") for link in links]))

    print(f"Se encontraron {len(video_page_urls)} páginas de videos. Extrayendo enlaces de YouTube...")

    youtube_ids = []
    for video_url in video_page_urls:
        try:
            driver.get(video_url)
            time.sleep(3)
            iframe = driver.find_element(By.CSS_SELECTOR, 'iframe[src*="youtube.com/embed"]')
            youtube_src = iframe.get_attribute('src')
            video_id = youtube_src.split("/embed/")[-1].split("?")[0]
            titulo = driver.title
            youtube_ids.append((titulo, video_id, video_url))
            print(f"✔ {titulo} - ID: {video_id}")
        except Exception as e:
            print(f"✘ No se encontró video de YouTube en {video_url}: {e}")
    return youtube_ids


from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound
from langdetect import detect

def clean_filename(filename):
    """Limpia un string para usarlo como nombre de archivo"""
    # Reemplazar caracteres no válidos para nombres de archivo
    clean = re.sub(r'[\\/*?:"<>|]', "", filename)
    # Limitar la longitud para evitar problemas con rutas demasiado largas
    if len(clean) > 150:
        clean = clean[:150]
    return clean

def get_transcripts_to_separate_files(video_list, output_dir="."):
    """Guarda cada transcripción en un archivo individual con el título del video"""
    # Crear directorio si no existe
    os.makedirs(output_dir, exist_ok=True)

    successful_transcripts = 0
    failed_transcripts = 0

    for titulo, video_id, url in video_list:
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['es', 'en'])
            text = "\n".join([seg["text"] for seg in transcript])
            detected_lang = detect(text)

            # Crear un nombre de archivo seguro basado en el título
            safe_title = clean_filename(titulo)
            filename = f"{output_dir}/{safe_title}.txt"

            with open(filename, "w", encoding="utf-8") as f:
                f.write(
                    f"TÍTULO: {titulo}\n"
                    f"ID: {video_id}\n"
                    f"URL: {url}\n"
                    f"IDIOMA DETECTADO: {detected_lang}\n"
                    f"TRANSCRIPCIÓN:\n{text}\n"
                )

            successful_transcripts += 1
            print(f"✔ Transcripción guardada para video: {titulo} (Idioma: {detected_lang})")
            print(f"  Archivo: {filename}")

        except (TranscriptsDisabled, NoTranscriptFound):
            failed_transcripts += 1
            print(f"✘ No se pudo obtener transcripción para video: {titulo}")
        except Exception as e:
            failed_transcripts += 1
            print(f"⚠ Error inesperado con el video {titulo}: {str(e)}")

    print(f"\nResumen: {successful_transcripts} transcripciones guardadas, {failed_transcripts} fallidas")
    return successful_transcripts, failed_transcripts

In [18]:
# Ejecutar todo
driver = start_driver(url_videos)
# Extraer todos los video_ids desde páginas internas
video_info_list = get_youtube_video_ids_from_bgg(driver, url_videos)
# Cerrar Selenium
driver.quit()
# Transcribir vídeos y guardarlos en archivos individuales
output_directory = "/content/transcripciones"  # Directorio para guardar los archivos
successful, failed = get_transcripts_to_separate_files(video_info_list, output_directory)
print(f"Proceso completado. Se guardaron {successful} transcripciones y fallaron {failed}.")

Esperando 5 segundos para que cargue la página...
Cargando enlaces internos de videos...
Se encontraron 37 páginas de videos. Extrayendo enlaces de YouTube...
✔ Sagrada - A "Chit" Chat Review by Amanda and Amanda | Video | BoardGameGeek - ID: jLCThJFmWo4
✔ The Art, Design, and Player Experience of Sagrada by Floodgate Games | Video | BoardGameGeek - ID: 4UUxnrQ2w14
✔ Sagrada — Gen Con 2016 | Video | BoardGameGeek - ID: GMeu2GbirqI
✔ Sagrada - Play Through, by Watch It Played | Video | BoardGameGeek - ID: w6mWCDnfLwc
✔ JonGetsGames - Sagrada Full Playthrough | Video | BoardGameGeek - ID: 3dzhP7Ol_WY
✔ Board to Death Video (6 min.) | Video | BoardGameGeek - ID: PvU9SuWU02I
✔ Sagrada Review - with Tom Vasel | Video | BoardGameGeek - ID: kL-seAlTBW8
✔ Bits of Board - Sagrada Review | Video | BoardGameGeek - ID: ib0etodHNS0
✔ The Game Boy Geek's Allegro (2-min) Review of Sagrada | Video | BoardGameGeek - ID: iZJyGePr6c8
✔ Bryan Drake @TheLatestRetro reviews Sagrada | Video | BoardGameGeek -

## Textos de pdf y docx

In [22]:
!pip install pdfplumber
!pip install python-docx

  Using cached pdfplumber-0.11.6-py3-none-any.whl.metadata (42 kB)
  Using cached pdfminer_six-20250327-py3-none-any.whl.metadata (4.1 kB)
  Using cached pypdfium2-4.30.1-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (48 kB)
Using cached pdfplumber-0.11.6-py3-none-any.whl (60 kB)
Using cached pdfminer_six-20250327-py3-none-any.whl (5.6 MB)
Using cached pypdfium2-4.30.1-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.9 MB/s eta 0:00:00


In [23]:
# Directorio de salida (mismo que para las transcripciones de video)
output_directory = "/content/transcripciones"
os.makedirs(output_directory, exist_ok=True)

# Lista de URLs de GitHub
github_urls = [
    "https://github.com/GrimaldiDamian/sagrada/blob/main/codigo/docx_pdfs/SAGRADA.pdf",
    "https://github.com/GrimaldiDamian/sagrada/blob/main/codigo/docx_pdfs/Sagrada-Rules-Floodgate-Games-SA01.pdf",
    "https://github.com/GrimaldiDamian/sagrada/blob/main/codigo/docx_pdfs/Sagrada__Automa_wPassion_-Deluxe_-_German.pdf",
    "https://github.com/GrimaldiDamian/sagrada/blob/main/codigo/docx_pdfs/Sagrada_solitaire_variant.docx"
]

import requests
import io
import pdfplumber
from docx import Document

def clean_filename(filename):
    """Limpia un string para usarlo como nombre de archivo"""
    # Reemplazar caracteres no válidos para nombres de archivo
    clean = re.sub(r'[\\/*?:"<>|]', "", filename)
    # Limitar la longitud para evitar problemas con rutas demasiado largas
    if len(clean) > 150:
        clean = clean[:150]
    return clean

def get_raw_content_url(github_url):
    """Convierte una URL de GitHub en la URL de contenido raw"""
    # Convertir la URL de GitHub a la URL de raw content
    raw_url = github_url.replace("github.com", "raw.githubusercontent.com")
    raw_url = raw_url.replace("/blob/", "/")
    return raw_url

def download_file(url):
    """Descarga un archivo desde una URL y devuelve su contenido como bytes"""
    raw_url = get_raw_content_url(url)
    print(f"Descargando archivo desde: {raw_url}")
    response = requests.get(raw_url)
    if response.status_code == 200:
        return response.content
    else:
        print(f"Error al descargar el archivo: {response.status_code}")
        return None

def extraer_texto_pdf(contenido_bytes):
    """Extrae texto de un PDF a partir de sus bytes"""
    texto = ""
    try:
        with pdfplumber.open(io.BytesIO(contenido_bytes)) as pdf:
            for pagina in pdf.pages:
                extracted_text = pagina.extract_text()
                if extracted_text:
                    texto += extracted_text + "\n"
        return texto
    except Exception as e:
        print(f"Error al procesar PDF: {e}")
        return ""

def extraer_texto_docx(contenido_bytes):
    """Extrae texto de un DOCX a partir de sus bytes"""
    try:
        doc = Document(io.BytesIO(contenido_bytes))
        return "\n".join([p.text for p in doc.paragraphs])
    except Exception as e:
        print(f"Error al procesar DOCX: {e}")
        return ""

def procesar_archivos():
    """Procesa cada archivo de la lista y guarda su contenido en un archivo de texto"""
    successful = 0
    failed = 0

    for url in github_urls:
        # Obtener el nombre del archivo de la URL
        filename = url.split("/")[-1]
        print(f"\nProcesando: {filename}")

        try:
            # Descargar el archivo
            contenido = download_file(url)
            if not contenido:
                print(f"✘ No se pudo descargar: {filename}")
                failed += 1
                continue

            # Extraer texto según el tipo de archivo
            if filename.lower().endswith(".pdf"):
                texto = extraer_texto_pdf(contenido)
            elif filename.lower().endswith(".docx"):
                texto = extraer_texto_docx(contenido)
            else:
                print(f"✘ Formato no soportado: {filename}")
                failed += 1
                continue

            if not texto:
                print(f"✘ No se pudo extraer texto de: {filename}")
                failed += 1
                continue

            # Crear nombre de archivo de salida
            name_without_extension = os.path.splitext(filename)[0]
            output_filename = clean_filename(name_without_extension) + ".txt"
            output_path = os.path.join(output_directory, output_filename)

            # Guardar texto en archivo individual
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(f"ARCHIVO ORIGINAL: {filename}\n")
                f.write(f"URL: {url}\n")
                f.write(f"CONTENIDO:\n\n{texto}\n")

            successful += 1
            print(f"✔ Texto extraído y guardado: {output_path}")

        except Exception as e:
            print(f"⚠ Error inesperado con {filename}: {str(e)}")
            failed += 1

    print(f"\nResumen: {successful} archivos procesados correctamente, {failed} fallidos")
    return successful, failed


In [24]:
# Ejecutar el procesamiento
print("Iniciando procesamiento de archivos desde GitHub...")
successful, failed = procesar_archivos()
print(f"Proceso completado. Se procesaron {successful} archivos y fallaron {failed}.")

Iniciando procesamiento de archivos desde GitHub...

Procesando: SAGRADA.pdf
Descargando archivo desde: https://raw.githubusercontent.com/GrimaldiDamian/sagrada/main/codigo/docx_pdfs/SAGRADA.pdf


✔ Texto extraído y guardado: /content/transcripciones/SAGRADA.txt

Procesando: Sagrada-Rules-Floodgate-Games-SA01.pdf
Descargando archivo desde: https://raw.githubusercontent.com/GrimaldiDamian/sagrada/main/codigo/docx_pdfs/Sagrada-Rules-Floodgate-Games-SA01.pdf
✔ Texto extraído y guardado: /content/transcripciones/Sagrada-Rules-Floodgate-Games-SA01.txt

Procesando: Sagrada__Automa_wPassion_-Deluxe_-_German.pdf
Descargando archivo desde: https://raw.githubusercontent.com/GrimaldiDamian/sagrada/main/codigo/docx_pdfs/Sagrada__Automa_wPassion_-Deluxe_-_German.pdf


✔ Texto extraído y guardado: /content/transcripciones/Sagrada__Automa_wPassion_-Deluxe_-_German.txt

Procesando: Sagrada_solitaire_variant.docx
Descargando archivo desde: https://raw.githubusercontent.com/GrimaldiDamian/sagrada/main/codigo/docx_pdfs/Sagrada_solitaire_variant.docx
✔ Texto extraído y guardado: /content/transcripciones/Sagrada_solitaire_variant.txt

Resumen: 4 archivos procesados correctamente, 0 fallidos
Proceso completado. Se procesaron 4 archivos y fallaron 0.


## Textos de página misutmeeple

In [25]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse

def clean_filename(filename):
    """Limpia un string para usarlo como nombre de archivo"""
    # Reemplazar caracteres no válidos para nombres de archivo
    clean = re.sub(r'[\\/*?:"<>|]', "", filename)
    # Limitar la longitud para evitar problemas con rutas demasiado largas
    if len(clean) > 150:
        clean = clean[:150]
    return clean

def extraer_contenido_resena(url):
    """
    Extrae contenido estructurado de una reseña de juego de mesa.
    Devuelve una lista con líneas de texto extraídas y un diccionario con las URLs de imágenes.
    """
    resultados = []  # Lista para acumular el contenido extraído
    imagenes_urls = []  # Lista para acumular URLs de imágenes

    # 1. Obtener el HTML de la página
    try:
        response = requests.get(url)
        if response.status_code == 200:
            html_content = response.text
            resultados.append("✅ Página obtenida con éxito\n")
        else:
            resultados.append(f"❌ Error al acceder a la página: {response.status_code}\n")
            return resultados, imagenes_urls
    except Exception as e:
        resultados.append(f"❌ Error al acceder a la página: {str(e)}\n")
        return resultados, imagenes_urls

    # 2. Crear objeto BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # 3. Título y encabezados
    try:
        titulo = soup.find("title").text.strip()
        resultados.append(f"Título de la pestaña: {titulo}\n")
    except:
        resultados.append("❌ No se encontró el título de la página\n")
        titulo = "sin_titulo"

    try:
        heading = soup.find("h1").text.strip()
        resultados.append(f"H1: {heading}\n")
    except:
        resultados.append("❌ No se encontró el encabezado H1\n")

    for nivel, tag in zip(["H2", "H3", "H4"], ["h2", "h3", "h4"]):
        encabezados = soup.find_all(tag)
        if encabezados:
            resultados.append(f"\n{nivel} encontrados:\n")
            for i in encabezados:
                resultados.append(f"- {i.text.strip()}")

    # 4. Párrafos
    parrafos = soup.find_all("p")
    if parrafos:
        resultados.append("\n\nPárrafos:\n")
        for i, parrafo in enumerate(parrafos, 1):
            texto = parrafo.text.strip()
            if texto:
                resultados.append(f"{i}. {texto}")

    # 5. Palabras en negrita
    strongs = soup.find_all("strong")
    if strongs:
        resultados.append("\n\nPalabras en negrita:\n")
        for i in strongs:
            texto = i.text.strip()
            if texto:
                resultados.append(f"- {texto}")

    # 6. Imágenes
    resultados.append("\n\nURLs de Imágenes encontradas:\n")
    try:
        content_wrap = soup.find(class_="entry-content-wrap") or soup.find("article") or soup
        imagenes = content_wrap.find_all("img")
        for i, img in enumerate(imagenes, 1):
            img_url = img.get("src")
            if img_url:
                if img_url.startswith("/"):
                    parsed_url = urlparse(url)
                    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
                    img_url = base_url + img_url
                resultados.append(f"Imagen {i}: {img_url}")
                imagenes_urls.append((f"imagen_{i}.jpg", img_url))
    except Exception as e:
        resultados.append(f"❌ Error al extraer imágenes: {str(e)}\n")

    # 7. Comentarios
    comentarios = soup.select(".comment-body")
    if comentarios:
        resultados.append("\n\nComentarios:\n")
        for i, comentario in enumerate(comentarios, 1):
            try:
                autor = comentario.find(class_="fn").text.strip()
                texto = comentario.find(class_="comment-content").text.strip()
                resultados.append(f"{i}. AUTOR: {autor}\nCOMENTARIO: {texto}\n")
            except:
                continue

    # 8. Listas
    elementos_lista = soup.find_all("ul") + soup.find_all("ol")
    if elementos_lista:
        resultados.append("\n\nElementos de listas:\n")
        for ul in elementos_lista:
            lista_li = ul.find_all("li")
            for li in lista_li:
                resultados.append(f"- {li.text.strip()}")

    # 9. Leyendas de imágenes
    leyendas = soup.find_all("figcaption")
    if leyendas:
        resultados.append("\n\nLeyendas de imágenes:\n")
        for i in leyendas:
            texto = i.text.strip()
            if texto:
                resultados.append(f"- {texto}")

    return resultados, imagenes_urls, titulo

def procesar_multiples_urls(urls, output_directory="/content/transcripciones"):
    """
    Procesa múltiples URLs, extrae su contenido y guarda cada una en un archivo de texto separado.
    También descarga las imágenes asociadas a cada URL en carpetas dedicadas.
    """
    # Crear directorio de salida si no existe
    os.makedirs(output_directory, exist_ok=True)

    successful = 0
    failed = 0

    for url in urls:
        print(f"\nProcesando URL: {url}")
        try:
            # Extraer contenido
            contenido, imagenes_urls, titulo = extraer_contenido_resena(url)

            if not contenido:
                print(f"✘ No se pudo extraer contenido de: {url}")
                failed += 1
                continue

            # Crear nombre de archivo para esta URL
            domain = urlparse(url).netloc.replace("www.", "")
            path = urlparse(url).path.strip("/").replace("/", "_")
            if path == "":
                path = "home"

            safe_filename = clean_filename(f"{domain}_{path}")
            txt_filename = f"{safe_filename}.txt"
            output_path = os.path.join(output_directory, txt_filename)

            # Guardar contenido en archivo
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(f"URL ORIGINAL: {url}\n")
                f.write(f"TÍTULO: {titulo}\n\n")
                f.write("\n".join(contenido))

            # Crear carpeta para imágenes si hay alguna
            if imagenes_urls:
                img_folder = os.path.join(output_directory, f"imagenes_{safe_filename}")
                os.makedirs(img_folder, exist_ok=True)

                # Descargar imágenes (limitado a las primeras 5)
                for i, (img_name, img_url) in enumerate(imagenes_urls[:5], 1):
                    try:
                        print(f"  Descargando imagen {i}/{min(5, len(imagenes_urls))}: {img_url}")
                        img_data = requests.get(img_url, timeout=10).content
                        img_path = os.path.join(img_folder, img_name)
                        with open(img_path, "wb") as f:
                            f.write(img_data)
                        # Pequeña pausa
                        time.sleep(1)
                    except Exception as e:
                        print(f"  ✘ Error al descargar imagen {i}: {str(e)}")

            successful += 1
            print(f"✔ Contenido extraído y guardado: {output_path}")
            if imagenes_urls:
                print(f"✔ Imágenes guardadas en: {img_folder}")

        except Exception as e:
            print(f"⚠ Error inesperado con {url}: {str(e)}")
            failed += 1

    print(f"\nResumen: {successful} URLs procesadas correctamente, {failed} fallidas")
    return successful, failed

In [28]:
# Lista de URLs a procesar
urls_a_procesar = [
    "https://misutmeeple.com/2018/08/resena-sagrada/"
]

output_directory = "/content/transcripciones"

# Ejecutar el script
print("Iniciando el scraping de múltiples URLs de reseñas de juegos...")
successful, failed = procesar_multiples_urls(urls_a_procesar, output_directory)
print(f"Proceso completado. Se procesaron {successful} URLs y fallaron {failed}.")

Iniciando el scraping de múltiples URLs de reseñas de juegos...

Procesando URL: https://misutmeeple.com/2018/08/resena-sagrada/
  Descargando imagen 1/5: https://i0.wp.com/misutmeeple.com/wp-content/uploads/2018/08/sagrada_portada.jpg?resize=1200%2C801&ssl=1
  Descargando imagen 2/5: https://i0.wp.com/misutmeeple.com/wp-content/uploads/2018/08/sagrada_contraportada.jpg?resize=1200%2C801&ssl=1
  Descargando imagen 3/5: https://i0.wp.com/misutmeeple.com/wp-content/uploads/2018/08/sagrada_contenido.jpg?resize=1200%2C394&ssl=1
  Descargando imagen 4/5: https://i0.wp.com/misutmeeple.com/wp-content/uploads/2018/08/sagrada_dados.jpg?resize=1200%2C246&ssl=1
  Descargando imagen 5/5: https://i0.wp.com/misutmeeple.com/wp-content/uploads/2018/08/sagrada_tablero_vidriera.jpg?resize=1200%2C823&ssl=1
✔ Contenido extraído y guardado: /content/transcripciones/misutmeeple.com_2018_08_resena-sagrada.txt
✔ Imágenes guardadas en: /content/transcripciones/imagenes_misutmeeple.com_2018_08_resena-sagrada

R

## Estadísticas

In [32]:
import pandas as pd

def start_driver(url: str, delay: int = 5) -> webdriver.Chrome:
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(delay)
    return driver

def guardar_csv(df: pd.DataFrame, output_dir: str, nombre_base: str):
    os.makedirs(output_dir, exist_ok=True)
    path_csv = os.path.join(output_dir, f"{nombre_base}.csv")
    df.to_csv(path_csv, sep=';', index=False)
    print(f"✅ CSV guardado: {path_csv}")

def obtener_stats(url: str, delay: int = 5) -> pd.DataFrame:
    driver = None
    try:
        print(f"Accediendo a {url}...")
        driver = start_driver(url, delay)

        stats = driver.find_element(By.CLASS_NAME, "global-body-content-primary.ng-scope")
        titulos = stats.find_elements(By.CLASS_NAME, "panel-title")
        titulos = [t for t in titulos if t.tag_name == "h3"]
        titulos_texto = [t.text for t in titulos]
        paneles = stats.find_elements(By.CLASS_NAME, "panel-body")

        datos = {}
        for titulo, panel in zip(titulos_texto, paneles):
            if titulo != "RATINGS BREAKDOWN":
                items = panel.find_elements(By.CLASS_NAME, "outline-item")
                valores = [item.text.replace("\n", " ") for item in items]
                datos[titulo] = valores

        return pd.DataFrame({k: pd.Series(v) for k, v in datos.items()})
    except Exception as e:
        print(f"❌ Error: {e}")
        return pd.DataFrame()
    finally:
        if driver:
            driver.quit()

def procesar_estadisticas(urls_dict, output_dir="/content/estadisticas"):
    os.makedirs(output_dir, exist_ok=True)
    for nombre_juego, url in urls_dict.items():
        print(f"\nProcesando: {nombre_juego}")
        df = obtener_stats(url)
        if not df.empty:
            guardar_csv(df, output_dir, f"estadisticas_{nombre_juego.lower().replace(' ', '_')}")
        else:
            print(f"❌ No se pudieron obtener estadísticas para {nombre_juego}")

In [33]:
# Diccionario con URLs
urls_estadisticas = {
    "Sagrada": "https://boardgamegeek.com/boardgame/199561/sagrada/stats",
    "Sagrada_5-6_Player": "https://boardgamegeek.com/boardgameexpansion/232199/sagrada-5-6-player-expansion/stats",
    "Sagrada_Great_Facades": "https://boardgamegeek.com/boardgameexpansion/293768/sagrada-great-facades-passion/stats"
}

# Ejecutar
print("Iniciando extracción...")
procesar_estadisticas(urls_estadisticas)
print("Proceso finalizado.")

Iniciando extracción...

Procesando: Sagrada
Accediendo a https://boardgamegeek.com/boardgame/199561/sagrada/stats...
✅ CSV guardado: /content/estadisticas/estadisticas_sagrada.csv

Procesando: Sagrada_5-6_Player
Accediendo a https://boardgamegeek.com/boardgameexpansion/232199/sagrada-5-6-player-expansion/stats...
✅ CSV guardado: /content/estadisticas/estadisticas_sagrada_5-6_player.csv

Procesando: Sagrada_Great_Facades
Accediendo a https://boardgamegeek.com/boardgameexpansion/293768/sagrada-great-facades-passion/stats...
✅ CSV guardado: /content/estadisticas/estadisticas_sagrada_great_facades.csv
Proceso finalizado.
